<a href="https://colab.research.google.com/github/merucode/Stock/blob/11_stock_info/21_theme_report(dust).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STEP 0. Install Dependency

In [ ]:
!pip install pykrx

# STEP 1. Infomation about Issue

### Google 검색어 트랜드
* [Google 검색어 트랜드](https://trends.google.co.kr/trends/explore?date=2011-01-01%202023-10-24&geo=KR&q=%EB%AF%B8%EC%84%B8%EB%A8%BC%EC%A7%80&hl=ko)

### Issue(일자 뉴스 서울미세먼지수치)
* 2017-03-24 [미세먼지 ‘매우 나쁨’… 붉은 조명의 남산타워
](https://www.seoul.co.kr/news/newsView.php?id=20170118500151)(130)
* 2018-03-25 [전국 미세먼지 나쁨 예보, 노약자·임산부·기저질환자 더 주의](https://m.health.chosun.com/svc/news_view.html?contid=2018032301083)(140)
* 2019-03-05 [오늘 서울 관측사상 최악 초미세먼지 가능성…150㎍/㎥ 육박](https://www.yna.co.kr/view/AKR20190305081600004)(200)
* 2021-03-29 [서울 미세먼지 경보·초미세먼지 주의보 동시 발령](https://www.chosun.com/national/transport-environment/2021/03/29/KUG6O5YU6BHIFF3HVYE6ZI4VR4/)(300)
* 2023-01-08 [미세먼지 감옥에 대한민국 갇혔다](https://www.seoul.co.kr/news/newsView.php?id=20230109001012)(150)
* 2023-03-23 [오후부터 황사 유입…서울 등 서쪽 중심 미세먼지 ‘매우나쁨’](https://www.seoul.co.kr/news/newsView.php?id=20230323500018)(150)

### Reference Site
* [서울특별시 대기환경정보 미세먼지통계-일별](https://cleanair.seoul.go.kr/statistics/dayAverage)

# STEP 2. 종합 분석 Plot

### Step 2-1. Def

In [ ]:
from pykrx import stock           # 데이터 : 주식 데이터 전송
import datetime                   # 그래프 : 문자열 시간 전환
import matplotlib.pyplot as plt   # 그래프
%matplotlib inline

tickers_KOSDAQ = stock.get_market_ticker_list(market="KOSDAQ")
tickers_KOSPI = stock.get_market_ticker_list(market="KOSPI")
tickers = tickers_KOSDAQ + tickers_KOSPI
ticker_dict = dict()

for ticker in tickers:
    stock_name = stock.get_market_ticker_name(ticker)
    ticker_dict[stock_name] = ticker

def convert_ohlcv_format(df):
    df = df.reset_index()
    df = df[['날짜', '시가', '고가', '저가', '종가', '거래량']]
    df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
    return df

def plot_with_issue_date(start_date, end_date, stock_name_lst, ticker_dict, issue_date_lst,  year_lst):
    ### Get ohlcv
    ohlcv_kosdaq = stock.get_index_ohlcv(start_date, end_date, "2001")
    ohlcv_kosdaq = convert_ohlcv_format(ohlcv_kosdaq)

    ohlcv_lst = []
    for stock_name in stock_name_lst:
        ohlcv = stock.get_market_ohlcv(start_date, end_date, ticker_dict[stock_name])
        ohlcv = convert_ohlcv_format(ohlcv)
        ohlcv['ad_close'] = ohlcv['close'] / ohlcv_kosdaq['close']
        ohlcv_lst.append(ohlcv)

    ### Convert str date to datetime date
    issue_date_dt_lst = []    # issue_date
    for issue_date_str in issue_date_lst:
        issue_date_dt_lst.append(datetime.datetime.strptime(issue_date_str, '%Y-%m-%d'))

    year_dt_lst = []    # year
    for year_str in year_lst:
        year_dt_lst.append(datetime.datetime.strptime(year_str, '%Y-%m-%d'))


    ### Plot
    fig, ax = plt.subplots(len(stock_name_lst)+1, 2, figsize=(14, 4+4*len(stock_name_lst)), constrained_layout=True)
    # figsize : 가로 10, 세로 stock_name_lst 갯수
    # constrained_layout=True : 서브플롯 간의 간격 자동 조절

    plot_color_lst = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']

    for ohlcv, stock_name in zip(ohlcv_lst, stock_name_lst):     # close price line plot
        ax[0][0].plot(ohlcv['date'], ohlcv['close']/ohlcv['close'][0], label=stock_name)
        ax[0][0].set_title('종합 그래프')

        ax[0][1].plot(ohlcv['date'], ohlcv['ad_close']/ohlcv['ad_close'][0], label=stock_name)
        ax[0][0].set_title('종합 그래프 adjusted by KOSDAQ')

    for i, ohlcv in enumerate(ohlcv_lst):     # close price adjust by KOSDAQ line plot
        ax[i+1][0].plot(ohlcv['date'], ohlcv['close'], color=plot_color_lst[i])
        ax[i+1][0].set_title(f'{stock_name_lst[i]}')

        ax[i+1][1].plot(ohlcv['date'], ohlcv['ad_close'], color=plot_color_lst[i])
        ax[i+1][1].set_title(f'{stock_name_lst[i]} adjusted by KOSDAQ')


    for pt in issue_date_dt_lst:    # add issue date line
        for i in range(0, len(stock_name_lst)+1):
            ax[i][0].axvline(x=pt, color='#D62728')
            ax[i][1].axvline(x=pt, color='#D62728')

    for pt in year_dt_lst:    # add year line
        for i in range(0, len(stock_name_lst)+1):
            ax[i][0].axvline(x=pt, color='#EFE034')
            ax[i][1].axvline(x=pt, color='#EFE034')

    ax[0][0].legend()
    ax[0][1].legend()
    plt.show()

### Step 2-2. Plot : '크린앤사이언스', '하츠', '위니아', '위닉스', '웰크론'

In [ ]:
start_date = '2017-01-01'
end_date = '2023-10-18'
stock_name_lst = ['크린앤사이언스', '하츠', '위니아', '위닉스', ]
# 1st Apply : '크린앤사이언스'(시총,반응), '하츠'(시총,반응), 위닉스(반응)
# 2nd Apply : '코웨이'(반응)
# Except : '모나리자'(반응小), '옵투스제약'(반응小), '웰크론'(반응小), '한독크린텍'(반응小), 위니아(적자)
# Wait :
year_lst = ['2018-01-01', '2019-01-01', '2020-01-01', '2021-01-01', '2022-01-01', '2023-01-01']
issue_date_lst = ['2017-03-24', '2018-03-25', '2019-03-05', '2021-03-29', '2023-01-08','2023-03-23']

plot_with_issue_date(start_date, end_date, stock_name_lst, ticker_dict, issue_date_lst, year_lst)

### Step 2-3. Adjust Plot

In [ ]:
plot_with_issue_date(start_date, end_date, stock_name_lst, ticker_dict, issue_date_lst, year_lst,  adjust=True)

# STEP 3. 개별분석

### Step 3-1. Def

In [ ]:
### 연도별 그래프
def plot_per_year(start_date, end_date, stock_name, ticker_dict, std_month):
    ### Get ohlcv
    ohlcv = stock.get_market_ohlcv(start_date, end_date, ticker_dict[stock_name])
    ohlcv = convert_ohlcv_format(ohlcv)
    ohlcv_kosdaq = stock.get_index_ohlcv(start_date, end_date, "2001")
    ohlcv_kosdaq = convert_ohlcv_format(ohlcv_kosdaq)
    ohlcv_kosdaq['close'] = ohlcv_kosdaq['close']
    ohlcv['ad_close'] = ohlcv['close'] / ohlcv_kosdaq['close']

    ### Plot
    y_lst = [2017, 2018, 2019, 2020, 2021, 2022, 2023]

    fig, ax = plt.subplots(1, 2, figsize=(10, 5), constrained_layout=True)
    for y in y_lst:
        filter = (f'{y+1}-{std_month}-01' > ohlcv['date']) & (ohlcv['date'] > f'{y}-{std_month}-01')
        ohlcv_y = ohlcv[filter]
        ohlcv_y = ohlcv_y.reset_index()

        ax[0].plot(ohlcv_y['close'], label=str(y))
        ax[1].plot(ohlcv_y['ad_close'], label=str(y))

    ax[0].set_title('종가 연도별 그래프')
    ax[1].set_title('수정종가 연도별 그래프')
    ax[0].legend()
    ax[1].legend()
    plt.show()

In [ ]:
def convert_cap_format(df):
    df = df.reset_index()
    df = df[['날짜', '시가총액', '상장주식수']]
    df.columns = ['date', 'cap', 'stocks']
    return df

### 상장주식 수
def plot_cap(start_date, end_date, stock_name, ticker_dict):
    ### Get cap data
    cap = stock.get_market_cap(start_date, end_date, ticker_dict[stock_name])
    cap = convert_cap_format(cap)

    fig, ax = plt.subplots(1, 2, figsize=(10, 5), constrained_layout=True)
    ax[0].plot(cap['date'], cap['cap'])
    ax[1].plot(cap['date'], cap['stocks'])

    ax[0].set_title('시가총액')
    ax[1].set_title('상장주식수')
    plt.show()

### 하츠

In [ ]:
start_date = '2017-01-01'
end_date = '2023-10-15'
stock_name = '하츠'
std_month = 10

plot_per_year(start_date, end_date, stock_name, ticker_dict, std_month)

In [ ]:
plot_cap(start_date, end_date, stock_name, ticker_dict)

# STEP 4. Report


### Issue

### Related Stock
*

### Plan
* Enter
  * 10~12월
* Exit
  * 1~3월 : 50% 7500 1/2